In [161]:
print("Welcome to Gen AI Demo at Tarush 2.0")


Welcome to Gen AI Demo at Tarush 2.0


In [162]:
import os
import boto3

boto_session = boto3.session.Session()
region = boto_session.region_name

boto_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=region,
    aws_access_key_id=os.environ.get("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY"),
    aws_session_token=os.environ.get("AWS_SESSION_TOKEN"),
)

In [163]:
# Load the dataset taken from kaggle.com

import pandas as pd

df = pd.read_csv("data/Global_AI_Content_Impact_Dataset.csv")
df

,Country,Year,Industry,AI Adoption Rate (%),AI-Generated Content Volume (TBs per year),Job Loss Due to AI (%),Revenue Increase Due to AI (%),Human-AI Collaboration Rate (%),Top AI Tools Used,Regulation Status,Consumer Trust in AI (%),Market Share of AI Companies (%)
0,South Korea,2022,Media,44.29,33.09,16.77,46.12,74.79,Bard,Strict,40.77,18.73
1,China,2025,Legal,34.75,66.74,46.89,52.46,26.17,DALL-E,Strict,35.67,35.02
2,USA,2022,Automotive,81.06,96.13,10.66,45.60,39.66,Stable Diffusion,Moderate,54.47,22.76
3,France,2021,Legal,85.24,93.76,27.70,78.24,29.45,Claude,Moderate,51.84,1.93
4,France,2021,Gaming,78.95,45.62,17.45,1.05,21.70,Midjourney,Strict,41.77,21.41
...,...,...,...,...,...,...,...,...,...,...,...,...
195,Germany,2021,Automotive,89.44,52.98,48.47,12.14,30.60,DALL-E,Lenient,77.21,44.35
196,Germany,2020,Media,70.11,28.26,27.62,57.86,58.71,DALL-E,Strict,78.74,31.73
197,France,2023,Marketing,65.77,49.83,39.94,79.44,43.73,Synthesia,Lenient,81.58,14.62
198,Australia,2023,Automotive,45.35,20.49,33.21,50.50,41.73,Claude,Strict,47.42,43.11


In [164]:
# Engineer Prompt

prompt_msg = "Write a summary of AI content impact for the given country referring the context."
country = "USA"
context = df.to_csv(index=False)

prompt = f"""
{prompt_msg}

Country: {country}

Global AI content impact details in CSV with headers:
{context}
"""


In [165]:
import json

llm_request_body = json.dumps(
    {
        "schemaVersion": "messages-v1",
        "messages": [
            {
                "role": "user",
                "content": [{"text": prompt}]
            }
        ],
        "inferenceConfig": {
            "maxTokens": 1024,      # The maximum number of tokens to allow in the generated response.
            "temperature": 0.3,     # Low: Conservative, High: creative
            "topP": 0.95,           # Balanced (0.9-0.95): Diverse and Coherence
            "topK": 10              # Low (5-20): Focused, High (20-50): More varied
        },
    }
)

In [166]:
response = boto_client.invoke_model(
    body=llm_request_body,
    modelId="apac.amazon.nova-micro-v1:0",  # Nova Micro model ID
    accept="application/json",
    contentType="application/json",
)
response_body = json.loads(response.get("body").read())

# Nova models return response in a different format
summary = response_body["output"]["message"]["content"][0]["text"]
print(summary)

In the USA, AI has shown significant adoption across various industries, particularly in automotive, retail, and manufacturing sectors. The AI adoption rate has been notably high, especially in the automotive industry where it reached 81.06% in 2022. This has led to a substantial volume of AI-generated content, with 96.13 TBs per year in the automotive sector in the same year. The impact on job loss due to AI has been moderate, with percentages varying across different sectors, such as 10.66% in automotive and 26.28% in retail by 2023. Conversely, the revenue increase due to AI has been significant, ranging from 22.58% in finance to 73.93% in retail by 2023. Human-AI collaboration rates have also been high, reflecting the integration of AI tools like Stable Diffusion and Synthesia across various sectors. The regulation status of AI in the USA is generally moderate, balancing innovation with consumer trust, which stands at 54.47% in 2022. The market share of AI companies in the USA has 

In [168]:
print("Thank You!!")

Thank You!!
